In [1]:
import signal, auxiliary, channel_func, math, warnings, sys, os
import threading, concurrent.futures
import random as rm
from threading import Lock

from typing import Any, Dict, Optional
from pathlib import Path
import json
import numpy as np
from sage.all import log, infinity, RR, ZZ, next_prime
from sage.graphs.generators.distance_regular import codes
from sage.matrix.constructor import matrix
from sage.modules.free_module_element import vector
from sage.rings.finite_rings.finite_field_constructor import GF
from sage.coding.guruswami_sudan.gs_decoder import roth_ruckenstein_root_finder

from auxiliary import print_matrix_with_precision
from kv_insdel import simulate_one_iteration

In [5]:
# RS code parameters
q = 101
n, k = 100, 33

GF_q = GF(q, modulus='primitive')
primitive_element = GF_q.gen()
eval_pts = [primitive_element ** i for i in range(n)]
assert n == len(set(eval_pts))
eval_pts = np.random.permutation(eval_pts)

# C is a primitive Reed-Solomon code with its evaluation points randomly permuted
C = codes.GeneralizedReedSolomonCode(eval_pts, k)  # eval points, dimension
code_param = auxiliary.CodeParams(q, n, k, eval_pts)

In [8]:
# ----------------- Choose decoding parameters -----------------------
# Davey-MacKay channel parameters (insertion, deletion & substitution probabilities)
Pi, Pd, Ps = 0.01, 0.01, 0.0
channel_param = auxiliary.ChannelParams(Pi, Pd, Ps)

# number of received sequences
rM = 2

# list size
L = 5

# set up decoding variables
ti, td = round(ceil(Pi * n) * 1.8), round(ceil(Pd * n) * 1.8 )

transmit_prob = channel_func.get_transmit_probabilities(q, n + 20, n, channel_param.pI, channel_param.pD)
transmit_prob_rev = channel_func.get_transmit_probabilities(q, n + 20, n, channel_param.pI, channel_param.pD)

term1_vec, term1_vec_rev = [], []
for i in range(-td, ti + 1):
    term1_vec.append(auxiliary.get_term_vector(q, n, n + i, Pi, Pd, transmit_prob))
    term1_vec_rev.append(auxiliary.get_term_vector(q, n, n + i, Pi, Pd, transmit_prob_rev))



In [9]:
simulate_one_iteration(code_param, channel_param, C, L, transmit_prob, transmit_prob_rev,
                            term1_vec, term1_vec_rev, td, ti, rM)

transmitted: [10, 38, 21, 8, 97, 41, 30, 6, 34, 63, 69, 49, 71, 98, 38, 61, 27, 53, 67, 66, 82, 25, 58, 44, 29, 58, 35, 78, 74, 72, 0, 32, 67, 43, 96, 35, 54, 46, 40, 3, 64, 83, 20, 81, 25, 28, 64, 99, 70, 70, 60, 82, 56, 7, 41, 70, 74, 82, 99, 61, 30, 23, 88, 70, 92, 3, 37, 35, 20, 48, 14, 34, 99, 19, 4, 70, 71, 15, 9, 16, 30, 27, 55, 61, 25, 38, 33, 70, 54, 32, 51, 46, 28, 32, 12, 26, 17, 18, 8, 49]
received:
(10, 38, 21, 8, 97, 30, 6, 34, 63, 69, 49, 71, 98, 38, 61, 27, 53, 67, 66, 82, 25, 58, 17, 44, 29, 58, 35, 78, 74, 72, 0, 32, 67, 43, 96, 35, 54, 46, 40, 3, 64, 83, 20, 81, 25, 28, 64, 99, 70, 70, 60, 82, 56, 7, 41, 70, 74, 82, 99, 61, 30, 23, 88, 70, 92, 3, 37, 35, 20, 48, 14, 34, 99, 19, 4, 70, 71, 15, 9, 16, 30, 27, 55, 61, 25, 38, 33, 70, 54, 32, 51, 46, 32, 12, 26, 17, 18, 8, 49)
(10, 38, 21, 8, 97, 41, 30, 6, 34, 63, 69, 49, 71, 98, 38, 61, 27, 53, 67, 66, 82, 25, 58, 29, 58, 35, 78, 74, 72, 0, 32, 67, 43, 96, 35, 54, 46, 40, 3, 64, 83, 20, 81, 25, 28, 64, 99, 70, 70, 60, 

{'numIterations': 1, 'frameErrors': 0, 'bitErrors': 0, 'erasures': 0}